In [256]:
import pandas as pd
import gurobipy as gb
from gurobipy import *
import numpy as np
from vizlib import *

In [258]:
avail = pd.read_csv('availability.csv')
num_weeks = len(avail)
weeks = list(range(0,num_weeks))

path = list(range(0,5))
path_i = path
res = list(range(5,9))
res_i = list(range(0,4))
all = path + res

b_ideal = np.round(np.array([12/52, 12/52, 12/52, 5/52, 12/52, 12/52, 12/52, 12/52, 12/52]) * num_weeks)
n_ideal = b_ideal

m = gb.Model()

# Decision variables
b = m.addMVar(avail.shape, name='biopsies', vtype=GRB.BINARY)
n = m.addMVar(avail.shape, name='necropsies', vtype=GRB.BINARY)

# Define combined schedule variable
c = m.addMVar(avail.shape, name='combined', vtype=GRB.BINARY)
m.addConstrs((c[:,a] == b[:,a] + n[:,a] for a in all), name='combine_schedules')

# Each duty requires one pathologist
m.addConstrs((quicksum(b[w,path]) == 1 for w in weeks), name='b_p_req')
m.addConstrs((quicksum(n[w,path]) == 1 for w in weeks), name='n_p_req')

# Each duty can have a max of one resident
m.addConstrs((quicksum(b[w,res]) <= 1 for w in weeks), name='b_r_max')
m.addConstrs((quicksum(n[w,res]) <= 1 for w in weeks), name='n_r_max')

# Cannot work both duties or during time off
m.addConstrs((b[w,a] + n[w,a] <= avail.iloc[w,a] for w in weeks for a in all), name='availability')

# Penalize double booking - except in the case of N/B which is ideal
nn_cost = quicksum(n[w,a]*n[w+1,a] for w in weeks[:-1] for a in all)
bn_cost = quicksum(b[w,a]*n[w+1,a] for w in weeks[:-1] for a in all)
bb_cost = quicksum(b[w,a]*b[w+1,a] for w in weeks[:-1] for a in all)
# Weights based on number of weeks waiting for N labs to come back and then do them
# May need to be modified depending on who is doing the microscope samples
double_penalty = 3*nn_cost + 2*bn_cost + bb_cost

# Penalize triple booking - eventually favor triple booking with 3rd year resident
cons = m.addMVar((len(weeks)-1, len(all)), name='consecutive', vtype=gb.GRB.BINARY)
m.addConstrs((cons[w,a] == c[w,a]*c[w+1,a] for w in weeks[:-1] for a in all), name='consecutive')
triple_penalty = quicksum(cons[w,a]*c[w+2,a] for w in weeks[:-2] for a in all)

# Any streak containing N should be followed with no scheduled and availability REWARD: TO-DO?

# Even pairing penalty
n_pairs = m.addVars(5, 4, vtype=GRB.INTEGER)
b_pairs = m.addVars(5, 4, vtype=GRB.INTEGER)
m.addConstrs((n_pairs[ip,ir] == quicksum(n[w,p]*n[w,r] for w in weeks) for ip,p in enumerate(path) for ir,r in enumerate(res)), name='pairs')
m.addConstrs((b_pairs[ip,ir] == quicksum(b[w,p]*b[w,r] for w in weeks) for ip,p in enumerate(path) for ir,r in enumerate(res)), name='pairs')
n_pair_var = quicksum((n_pairs[p,r] - (n_pairs.sum()/len(n_pairs)))**2 for p in path_i for r in res_i)
b_pair_var = quicksum((b_pairs[p,r] - (b_pairs.sum()/len(b_pairs)))**2 for p in path_i for r in res_i)
pair_balance_penalty = n_pair_var + b_pair_var

# Penalize mismatch of number of duties per employee
n_d = m.addVars(len(all), vtype=GRB.INTEGER)
b_d = m.addVars(len(all), vtype=GRB.INTEGER)
m.addConstrs((n_d[a] == (quicksum(n[w,a] for w in weeks) - n_ideal[a])**2 for a in all), name='n_ideal_difference')
m.addConstrs((b_d[a] == (quicksum(b[w,a] for w in weeks) - b_ideal[a])**2 for a in all), name='b_ideal_difference')
work_balance_penalty = n_d.sum() + b_d.sum()

# Objective function
m.setObjective(50*triple_penalty + 5*double_penalty + 5*work_balance_penalty + pair_balance_penalty, gb.GRB.MINIMIZE)
m.optimize()

# Visualize the schedule with avail dataframe, b matrix and n matrix
viz_schedule(avail, b, n)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 462 rows, 805 columns and 1323 nonzeros
Model fingerprint: 0x7fc5a9a6
Model has 1131 quadratic objective terms
Model has 238 quadratic constraints
Variable types: 0 continuous, 805 integer (747 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+01]
  Objective range  [5e+00, 5e+00]
  QObjective range [2e-01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [4e+00, 2e+01]
Presolve removed 267 rows and 298 columns
Presolve time: 0.02s
Presolved: 7455 rows, 2879 columns, 20066 nonzeros
Presolved model has 1152 quadratic objective terms
Variable types: 0 continuous, 2879 integer (2821 binary)
Found heuristic solution: objective 3273.1000000
Found heuristic solution: objective 2349.0000000

Root relaxation: object

,NM,MP,SA,LKC,DG,MC,AK,CC,KB
0,,B,N,-,,,-,N,B
1,N,-,-,-,B,N,B,,-
2,,N,-,-,B,,B,N,-
3,B,-,-,-,N,N,,,B
4,B,N,-,-,,B,N,-,
5,,B,,-,N,,-,-,B
6,B,-,N,-,-,N,,B,
7,N,-,B,-,-,,N,,-
8,,B,-,N,-,N,B,,-
9,B,,-,-,N,,-,N,
